# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Importing neccessary Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
##connecting to the cluster
import cassandra
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect() # To establish connection and begin executing queries, need a session

except Exception as e:
    print(e)


#### Create Keyspace

In [6]:
# creating keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS apache_cas 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# setting up the kepspace 
try:
    session.set_keyspace('apache_cas')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [10]:
# create table
query = "CREATE TABLE IF NOT EXISTS song_info "
query = query + "(sessionId int, itemInSession int, artist varchar, length decimal, song varchar, Primary Key(sessionId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

### Inserting into table song_info

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## inserting necessary columns into the table
        query = "insert into song_info(sessionId,itemInSession, artist,length,song)"
        query = query + "values(%s, %s ,%s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0], float(line[5]), line[9]))

#### This query is to fetch artist, song, song length from song_info table

In [12]:
#querying to see if the query answered the question
query = "select artist, song, length from song_info where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist,"\nSong: " +row.song, "\nSong Length: "+str(row.length))

Artist: Faithless 
Song: Music Matters (Mark Knight Dub) 
Song Length: 495.3073


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [13]:
## Query - 2
## create table

query = "CREATE TABLE IF NOT EXISTS users_song_info "
query = query + "( userId int,sessionId int,  itemInSession int, artist varchar,firstName varchar, lastName varchar, song varchar, \
Primary Key((userId,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

### Inserting into table users_song_info

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## inserting necessary columns into the table
        query = "insert into users_song_info(userId,sessionId,itemInSession,artist,firstName,lastName,song)"
        query = query + "values(%s, %s ,%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]), line[0],line[1], line[4], line[9]))

### This query is to fetch artist, song, firstname and lastname from users_song_info table

In [15]:
#querying to see if the query answered the question
query_2 = "select artist, song, firstName, lastName  from users_song_info where userId=10 and sessionId = 182"
try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist, "\nSong: "+row.song,"\nFirst Name: "+row.firstname,"\nLast Name:" +row.lastname,"\n")

Artist: Down To The Bone 
Song: Keep On Keepin' On 
First Name: Sylvie 
Last Name:Cruz 

Artist: Three Drives 
Song: Greece 2000 
First Name: Sylvie 
Last Name:Cruz 

Artist: Sebastien Tellier 
Song: Kilometer 
First Name: Sylvie 
Last Name:Cruz 

Artist: Lonnie Gordon 
Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
First Name: Sylvie 
Last Name:Cruz 



### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
## Query - 3
## create table
query = "CREATE TABLE IF NOT EXISTS users_info "
query = query + "(song varchar,userId int, firstName varchar, lastName varchar ,Primary Key(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

### Inserting into table users_info

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## inserting necessary columns into the table
        query = "insert into users_info(song, userId, firstName,lastName )"
        query = query + "values(%s, %s ,%s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

### This query is to fetch first name , last name from the users_info table

In [18]:
#querying to see if the query answered the question
query_3 = "select firstName , lastName , userId from users_info where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)
    
for row in rows:
    print("First Name: "+row.firstname,"\nLast Name:" +row.lastname,"\n")

First Name: Jacqueline 
Last Name:Lynch 

First Name: Tegan 
Last Name:Levine 

First Name: Sara 
Last Name:Johnson 



### Drop the tables before closing out the sessions

In [19]:
##Drop the table before closing out the sessions
query = "drop table song_info"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [20]:
##Drop the table before closing out the sessions
query = "drop table users_song_info"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [21]:
##Drop the table before closing out the sessions
query = "drop table users_info"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()